### establish a percent change metric
rolling forward

In [254]:
!pip3 install matplotlib
!pip3 install altair

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [255]:
import altair as alt
import pandas as pd
import os
import numpy as np

In [256]:
!mv $(find . -type d -name "lob_caps" -exec grep -q MATCH {} \; -print0 | xargs -0 echo) backup_match/

grep: ./lob_caps: Is a directory
usage: mv [-f | -i | -n] [-hv] source target
       mv [-f | -i | -n] [-v] source ... directory


In [257]:
#https://stackoverflow.com/a/21232849 model 
def getCAPSByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
                                # print("for type, ", type)
    ret = []
    for root, dirs, files in os.walk("./lob_caps/"):
        for filename in files:
            if type in filename:
#                 print("CAPS file, ", filename) #mac, do find . -name ._\* -delete
                ret.append(filename)
    return ret

csvFileList = getCAPSByDateAndType("CAPS") #iterate this array to dip into each csv, later on
li = []                         #form the endFrame / global data frame around this array
for filename in csvFileList:
    csv = "lob_caps/" + filename
    # print(csv)
    df = pd.read_csv(csv, index_col=None, header=0)
    li.append(df)

capsFrame = pd.concat(li, axis=0, ignore_index=True) #end frame contains all data
capsFrame.sort_values(by=['time'], ascending=True)   #sorted by time into one time series
print("for new df: ", capsFrame.shape[0])
start = capsFrame["time"].min()
end = capsFrame["time"].max()
print("start: ", start, " end: ", end)
print(capsFrame.columns)

for new df:  284997
start:  1660221600292.0  end:  1691444880101.0
Index(['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid'], dtype='object')


In [258]:
def getSKEWByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
                                # print("for type, ", type)
    ret = []
    for root, dirs, files in os.walk("./lob_caps/"):
        for filename in files:
            if type in filename:
#                 print("CAPS file, ", filename) #mac, do find . -name ._\* -delete
                ret.append(filename)
    return ret

csvFileList = getSKEWByDateAndType("MEANSHIFT") #iterate this array to dip into each csv, later on
li = []                         #form the endFrame / global data frame around this array
for filename in csvFileList:
    csv = "lob_caps/" + filename
    # print(csv)
    df = pd.read_csv(csv, index_col=None, header=0)
    li.append(df)

skewFrame = pd.concat(li, axis=0, ignore_index=True) #end frame contains all data
skewFrame.sort_values(by=['timeStamp'], ascending=True)   #sorted by time into one time series
skewFrame.rename(columns={'timeStamp': 'time'}, inplace=True)
print("for new df: ", skewFrame.shape[0])
start = skewFrame["time"].min()
end = skewFrame["time"].max()
print("start: ", start, " end: ", end)
print(skewFrame.columns)

for new df:  139237
start:  1660221606227  end:  1691444966359
Index(['mp', 'time', 'mean', 'skew'], dtype='object')


In [259]:
# Merging those two data frames Will not take place based on a Shared key of time
# You'll need to look up the SKU value for every row based on approximation
# merged_df = pd.merge(capsFrame, skewFrame, on='time')

In [260]:
skewFrame.head(10)

,mp,time,mean,skew
0,17.66,1677651424290,5877.906080,-5860.246080
1,17.66,1677651427404,5906.691222,-5889.031222
2,17.65,1677651430373,5944.892609,-5927.242609
3,17.66,1677651436323,5950.952405,-5933.292405
4,17.66,1677651439323,5926.797221,-5909.137221
5,17.66,1677651442399,5909.721180,-5892.061180
6,17.66,1677651445381,5898.866475,-5881.206475
7,17.66,1677651448534,5899.116796,-5881.456796
8,17.66,1677651451564,5891.509682,-5873.849682
9,17.66,1677651454328,5891.185193,-5873.525193


### Apply skew and mean values from mean shift to the caps data frame FAIL

for the data frame capsFrame set the column of 'mean' equal to the data frame, skewFrame 'mean' where the 'time' column in both data frames is approximately equal to within one minute



In [261]:
# capsFrame.loc[capsFrame['time'].sub(skewFrame['time'].values[0]).abs().idxmin(), 'mean'] = skewFrame['mean'].values[0]
# capsFrame.loc[capsFrame['time'].sub(skewFrame['time'].values[0]).abs().idxmin(), 'skew'] = skewFrame['skew'].values[0]
# capsFrame.loc[capsFrame['time'].sub(skewFrame['time'].values[0]).abs().lt(pd.Timedelta(minutes=2)).idxmax(), 'mean'] = skewFrame['mean'].values[0]
# 
# Could not get this operation to work try it again using a range of values once the precursor and surge are defined


In [262]:
capsFrame.head(2)

,bc,ac,tbv,tav,time,mp,minBid
0,2435138.24,10557430.61,493817.73,244833.13,1.672418e+12,10.78,10.74
1,2430127.12,10557235.51,493352.24,244815.55,1.672418e+12,10.78,10.74


In [263]:


# Load your time series data into a pandas dataframe
caps_df = pd.read_csv('Fri Apr 07 2023 12:26:51 GMT-0700 (Pacific Daylight Time)-CAPS.csv')


lookback_period = 10 # in rows
caps_df['change'] = caps_df['mp'].pct_change(periods=lookback_period)
# caps_df.sample
print(caps_df.shape[0], caps_df.columns)# Calculate the returns of your asset over a fixed lookback period

926 Index(['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change'], dtype='object')


In [264]:
#for period, average or mean change metric. this changes with window size
meanChange = round(caps_df['change'].mean(),8)
meanChange

-3.996e-05

In [265]:
#reset mean chang ebery 24 hrs

In [266]:
# identify units of 10 rows where the percent change is greater or less than the threshold

threshold = meanChange
surges = []
precursors = []

for i in range(0,len(caps_df),10):

    if caps_df.iloc[i:i+10]['change'].mean() >= threshold:
        
        surges.append({'time': caps_df.iloc[i]['time'],
                       's_MP': caps_df.iloc[i]['mp'],
                       'change': caps_df.iloc[i:i+10]['change'].mean(),
                       'type':'surge'})  #['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change']
    else:
        precursors.append({'time': caps_df.iloc[i]['time'],
                           'p_MP': caps_df.iloc[i]['mp'],
                           'change': caps_df.iloc[i:i+10]['change'].mean(),
                        'type':'precursor',
                        'p_buyCap':caps_df.iloc[i]['bc'], 
                        'p_askCap':caps_df.iloc[i]['ac'],
                        'p_totalBidVol':caps_df.iloc[i]['tbv'],
                        'p_totalAskVol':caps_df.iloc[i]['tav']
                        })  

In [267]:
for item in surges:
    print(item)


{'time': 1680895735340.0, 's_MP': 17.59, 'change': 0.0, 'type': 'surge'}
{'time': 1680895765299.0, 's_MP': 17.59, 'change': 0.0, 'type': 'surge'}
{'time': 1680895795332.0, 's_MP': 17.59, 'change': 0.0, 'type': 'surge'}
{'time': 1680895826633.0, 's_MP': 17.59, 'change': 0.00039795338260384394, 'type': 'surge'}
{'time': 1680895863002.0, 's_MP': 17.6, 'change': 0.0001705514496873617, 'type': 'surge'}
{'time': 1680895935966.0, 's_MP': 17.6, 'change': 0.00011370096645824113, 'type': 'surge'}
{'time': 1680895992486.0, 's_MP': 17.6, 'change': 0.00017045454545452365, 'type': 'surge'}
{'time': 1680896047452.0, 's_MP': 17.61, 'change': 0.00039772727272722184, 'type': 'surge'}
{'time': 1680896203843.0, 's_MP': 17.6, 'change': 5.685044654391014e-05, 'type': 'surge'}
{'time': 1680896290548.0, 's_MP': 17.62, 'change': 0.0007384428851511471, 'type': 'surge'}
{'time': 1680896347965.0, 's_MP': 17.63, 'change': 0.0003972758229283446, 'type': 'surge'}
{'time': 1680896456634.0, 's_MP': 17.63, 'change': 0.

In [268]:
for item in precursors:
    print(item)

{'time': 1680895615420.0, 'p_MP': 17.61, 'change': nan, 'type': 'precursor', 'p_buyCap': 3537895.45, 'p_askCap': 32564599.64, 'p_totalBidVol': 417439.52, 'p_totalAskVol': 328777.66}
{'time': 1680895645356.0, 'p_MP': 17.61, 'change': -0.0003975014196478499, 'type': 'precursor', 'p_buyCap': 3522640.95, 'p_askCap': 32577586.24, 'p_totalBidVol': 416576.54, 'p_totalAskVol': 329515.14}
{'time': 1680895675317.0, 'p_MP': 17.6, 'change': -0.00017035775127764995, 'type': 'precursor', 'p_buyCap': 3519748.95, 'p_askCap': 32551526.13, 'p_totalBidVol': 416417.82, 'p_totalAskVol': 328048.27}
{'time': 1680895705306.0, 'p_MP': 17.59, 'change': -0.0005681818181818565, 'type': 'precursor', 'p_buyCap': 3512363.25, 'p_askCap': 32549903.68, 'p_totalBidVol': 416014.34, 'p_totalAskVol': 327997.32}
{'time': 1680895900177.0, 'p_MP': 17.6, 'change': -0.00011363636363637131, 'type': 'precursor', 'p_buyCap': 3510978.75, 'p_askCap': 32528767.96, 'p_totalBidVol': 415987.69, 'p_totalAskVol': 326772.43}
{'time': 16808

In [269]:
surges_df = pd.DataFrame(surges)
precursors_df = pd.DataFrame(precursors)
sequence_df = pd.concat([surges_df, precursors_df]).sort_values(by=['time'], ascending=True)


In [270]:
# for index, row in sequence_df.iterrows():
#     print(row['surge'], row['precursor'])
sequence_df['type'].head(40)

0     precursor
1     precursor
2     precursor
3     precursor
0         surge
1         surge
2         surge
3         surge
4         surge
4     precursor
5         surge
6         surge
7         surge
5     precursor
8         surge
9         surge
10        surge
11        surge
12        surge
13        surge
14        surge
6     precursor
7     precursor
15        surge
8     precursor
9     precursor
10    precursor
11    precursor
12    precursor
13    precursor
14    precursor
16        surge
17        surge
18        surge
15    precursor
16    precursor
19        surge
20        surge
17    precursor
18    precursor
Name: type, dtype: object

In [271]:
sequence_df.head(45)

,time,s_MP,change,type,p_MP,p_buyCap,p_askCap,p_totalBidVol,p_totalAskVol
0,1.680896e+12,NaN,NaN,precursor,17.61,3537895.45,32564599.64,417439.52,328777.66
1,1.680896e+12,NaN,-3.975014e-04,precursor,17.61,3522640.95,32577586.24,416576.54,329515.14
2,1.680896e+12,NaN,-1.703578e-04,precursor,17.60,3519748.95,32551526.13,416417.82,328048.27
3,1.680896e+12,NaN,-5.681818e-04,precursor,17.59,3512363.25,32549903.68,416014.34,327997.32
0,1.680896e+12,17.59,0.000000e+00,surge,NaN,NaN,NaN,NaN,NaN
1,1.680896e+12,17.59,0.000000e+00,surge,NaN,NaN,NaN,NaN,NaN
2,1.680896e+12,17.59,0.000000e+00,surge,NaN,NaN,NaN,NaN,NaN
3,1.680896e+12,17.59,3.979534e-04,surge,NaN,NaN,NaN,NaN,NaN
4,1.680896e+12,17.60,1.705514e-04,surge,NaN,NaN,NaN,NaN,NaN
4,1.680896e+12,NaN,-1.136364e-04,precursor,17.60,3510978.75,32528767.96,415987.69,326772.43


In [272]:

line = alt.Chart(sequence_df).mark_line(color='green').encode(
    x='time',
    y='s_MP'
)

s_bar = alt.Chart(sequence_df).mark_bar().encode(
    x='time',
    y='s_MP',
    color='type:N'
)

p_bar = alt.Chart(sequence_df).mark_bar().encode(
    x='time',
    y='p_MP',
    color='type:N'
)

chart = (line + s_bar + p_bar).properties(width=600, height=500)
chart.title = 'Sequential order of precursor and surges for April 7th 2023'

subtitle = 'Precursors are contiguous periods where percentage rate of growth is less than threshold'
chart.properties(title=alt.TitleParams(text=[chart.title, subtitle], baseline='bottom', orient='top', anchor='start', fontSize=14))
chart

alt.LayerChart(...)

In [273]:
sequence_df.columns

Index(['time', 's_MP', 'change', 'type', 'p_MP', 'p_buyCap', 'p_askCap',
       'p_totalBidVol', 'p_totalAskVol'],
      dtype='object')

### Grouping together bucketed surge items

Create one line to describe a precursor or search and it's related order book statistics

In [281]:

sequence_df['group'] = (sequence_df['type'] != sequence_df['type'].shift(1)).cumsum()
sequence_df['length'] = sequence_df.groupby(['type', 'group'])['group'].transform('count')
sequence_df['identifier'] = sequence_df.groupby(['type', 'group'])['time'].transform('min') #prep the label early, if surge?

sequence_df['sum_change'] = sequence_df.groupby(['type', 'group'])['change'].transform('sum')
sequence_df['area']  = sequence_df.apply(lambda row: row['length'] * row['sum_change'], axis=1)
sequence_df.loc[sequence_df['type'] == 'surge', 'surge_area'] = sequence_df.loc[sequence_df['type'] == 'surge', 'area']
sequence_df.columns
# sequence_df['area'] = sequence_df.groupby(['type', 'group'])['sum_change'].multiply(sequence_df['length'])
# sequence_df.loc[sequence_df['type'] == 'surge', 'surge_area'] =  sequence_df['length'] * sequence_df['sum_change']


# sequence_df['sum'] = sequence_df.groupby(['surge', 'group'])['change'].transform('sum')

# sequence_df['end_time'] = sequence_df.groupby(['surge', 'group'])['time'].transform('max')
# sequence_df['type'] = sequence_df['surge']

# sequence_df['buyCapSum'] = sequence_df.groupby(['surge', 'group'])['buyCap'].transform('avg')
# sequence_df['askCapSum'] = sequence_df.groupby(['surge', 'group'])['askCap'].transform('avg')

# calculate the area for the surge

# sequence_df = sequence_df.drop('next_value', axis=1)
# sequence_df.loc[sequence_df['bucket'] == 'surge', 'surge_length'] =  sequence_df['length']
# sequence_df.drop('length', axis=1, inplace=True)
# df = df.loc[:,~df.columns.duplicated()]

#unique_df = sequence_df.groupby('identifier').first().reset_index()

# unique_df.loc[unique_df['surge'] == '1', 'surge_length'] = unique_df['length']
# unique_df.loc[unique_df['surge'] == '0', 'length'] = 0

Index(['time', 's_MP', 'change', 'type', 'p_MP', 'p_buyCap', 'p_askCap',
       'p_totalBidVol', 'p_totalAskVol', 'group', 'length', 'identifier',
       'sum_change', 'area', 'surge_area'],
      dtype='object')

In [282]:
sequence_df.head(20)

,time,s_MP,change,type,p_MP,p_buyCap,p_askCap,p_totalBidVol,p_totalAskVol,group,length,identifier,sum_change,area,surge_area
0,1.680896e+12,NaN,NaN,precursor,17.61,3537895.45,32564599.64,417439.52,328777.66,1,4,1.680896e+12,-0.001136,-0.004544,NaN
1,1.680896e+12,NaN,-0.000398,precursor,17.61,3522640.95,32577586.24,416576.54,329515.14,1,4,1.680896e+12,-0.001136,-0.004544,NaN
2,1.680896e+12,NaN,-0.000170,precursor,17.60,3519748.95,32551526.13,416417.82,328048.27,1,4,1.680896e+12,-0.001136,-0.004544,NaN
3,1.680896e+12,NaN,-0.000568,precursor,17.59,3512363.25,32549903.68,416014.34,327997.32,1,4,1.680896e+12,-0.001136,-0.004544,NaN
0,1.680896e+12,17.59,0.000000,surge,NaN,NaN,NaN,NaN,NaN,2,5,1.680896e+12,0.000569,0.002843,0.002843
1,1.680896e+12,17.59,0.000000,surge,NaN,NaN,NaN,NaN,NaN,2,5,1.680896e+12,0.000569,0.002843,0.002843
2,1.680896e+12,17.59,0.000000,surge,NaN,NaN,NaN,NaN,NaN,2,5,1.680896e+12,0.000569,0.002843,0.002843
3,1.680896e+12,17.59,0.000398,surge,NaN,NaN,NaN,NaN,NaN,2,5,1.680896e+12,0.000569,0.002843,0.002843
4,1.680896e+12,17.60,0.000171,surge,NaN,NaN,NaN,NaN,NaN,2,5,1.680896e+12,0.000569,0.002843,0.002843
4,1.680896e+12,NaN,-0.000114,precursor,17.60,3510978.75,32528767.96,415987.69,326772.43,3,1,1.680896e+12,-0.000114,-0.000114,NaN


## Critical group by unique identifier

In [284]:
unique_df = sequence_df.groupby('identifier').first().reset_index()
print(unique_df)

      identifier          time   s_MP        change       type   p_MP  \
0   1.680896e+12  1.680896e+12    NaN -3.975014e-04  precursor  17.61   
1   1.680896e+12  1.680896e+12  17.59  0.000000e+00      surge    NaN   
2   1.680896e+12  1.680896e+12    NaN -1.136364e-04  precursor  17.60   
3   1.680896e+12  1.680896e+12  17.60  1.137010e-04      surge    NaN   
4   1.680896e+12  1.680896e+12    NaN -5.678592e-05  precursor  17.61   
5   1.680896e+12  1.680896e+12  17.60  5.685045e-05      surge    NaN   
6   1.680897e+12  1.680897e+12    NaN -3.968254e-04  precursor  17.64   
7   1.680897e+12  1.680897e+12  17.62  1.702933e-04      surge    NaN   
8   1.680898e+12  1.680898e+12    NaN -2.267573e-04  precursor  17.64   
9   1.680899e+12  1.680899e+12  17.57  3.882788e-07      surge    NaN   
10  1.680901e+12  1.680901e+12    NaN -1.987636e-03  precursor  17.59   
11  1.680902e+12  1.680902e+12  17.57  9.674936e-04      surge    NaN   
12  1.680903e+12  1.680903e+12    NaN -1.758302e-03

### Merge even and odd Rows

In [287]:
even_df = unique_df.iloc[::2].reset_index(drop=True)
odd_df = unique_df.iloc[1::2].reset_index(drop=True)

merged_df = pd.concat([even_df, odd_df], axis=1)

print(merged_df)

      identifier          time  s_MP    change       type   p_MP    p_buyCap  \
0   1.680896e+12  1.680896e+12   NaN -0.000398  precursor  17.61  3537895.45   
1   1.680896e+12  1.680896e+12   NaN -0.000114  precursor  17.60  3510978.75   
2   1.680896e+12  1.680896e+12   NaN -0.000057  precursor  17.61  3521737.66   
3   1.680897e+12  1.680897e+12   NaN -0.000397  precursor  17.64  3528630.16   
4   1.680898e+12  1.680898e+12   NaN -0.000227  precursor  17.64  3516764.02   
5   1.680901e+12  1.680901e+12   NaN -0.001988  precursor  17.59  3514346.80   
6   1.680903e+12  1.680903e+12   NaN -0.001758  precursor  17.62  3549763.56   
7   1.680906e+12  1.680906e+12   NaN -0.000568  precursor  17.60  3496964.66   
8   1.680909e+12  1.680909e+12   NaN -0.001075  precursor  17.67  3543523.20   
9   1.680921e+12  1.680921e+12   NaN -0.000626  precursor  17.56  3536670.70   
10  1.680926e+12  1.680926e+12   NaN -0.000169  precursor  17.60  3544877.07   
11  1.680939e+12  1.680939e+12   NaN -0.

In [293]:
nan_cols = merged_df.dropna(axis=1, how='all')
nan_cols.head()

,identifier,time,change,type,p_MP,p_buyCap,p_askCap,p_totalBidVol,p_totalAskVol,group,...,identifier,time,s_MP,change,type,group,length,sum_change,area,surge_area
0,1.680896e+12,1.680896e+12,-0.000398,precursor,17.61,3537895.45,32564599.64,417439.52,328777.66,1,...,1.680896e+12,1.680896e+12,17.59,0.000000e+00,surge,2.0,5.0,0.000569,0.002843,0.002843
1,1.680896e+12,1.680896e+12,-0.000114,precursor,17.60,3510978.75,32528767.96,415987.69,326772.43,3,...,1.680896e+12,1.680896e+12,17.60,1.137010e-04,surge,4.0,3.0,0.000682,0.002046,0.002046
2,1.680896e+12,1.680896e+12,-0.000057,precursor,17.61,3521737.66,32537607.79,416591.06,327169.45,5,...,1.680896e+12,1.680896e+12,17.60,5.685045e-05,surge,6.0,7.0,0.001760,0.012318,0.012318
3,1.680897e+12,1.680897e+12,-0.000397,precursor,17.64,3528630.16,32539119.92,416741.59,327356.38,7,...,1.680897e+12,1.680897e+12,17.62,1.702933e-04,surge,8.0,1.0,0.000170,0.000170,0.000170
4,1.680898e+12,1.680898e+12,-0.000227,precursor,17.64,3516764.02,32528948.22,416136.48,326788.49,9,...,1.680899e+12,1.680899e+12,17.57,3.882788e-07,surge,10.0,3.0,0.001593,0.004779,0.004779


In [294]:
nan_cols.to_csv('nan_cols.csv', index=False)
